# "US Presidential Candidate Tweets"
> "A visualization of recent tweets from candidates in the 2020 US Presidential Election"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [Data Visualization]
- image: 
- hide: false
- search_exclude: true

## Imports Modules

In [3]:
#collapse-hide
import altair as alt
import pandas as pd

## Import Data

In [7]:
def read_tweet_data(username, date):
  url = "https://raw.githubusercontent.com/joshkraft/daily-candidate-tweets/main/data/" + str(username) + "/" + str(date) + ".csv"
  df = pd.read_csv(url)
  return df

biden_tweets = read_tweet_data("JoeBiden", "2020-10-20")
trump_tweets = read_tweet_data("realDonaldTrump", "2020-10-20")

'Lesley Stahl of 60 Minutes not wearing a mask in the White House after her interview with me. Much more to come. https://t.co/0plZG6a4fH'

## Plot